In [4]:
import jieba
import re
import MySQLdb
from gensim.models import word2vec
import numpy as np
import pprint

db = MySQLdb.connect("slugchat-test.lorabit.com", "root", "password", "slugchat")
cursor = db.cursor()

ignore_words = {'的', '地', '了', '和', '是', '就', '都', '而', '及', '与', '着', '或',
              '一个', '没有', '我', '我们', '你', '你们', '他', '他们', '她', '她们', '它', '他们', '们', '是否'}

In [147]:
sql = "SELECT content FROM slugchat.tbl_stories"
cursor.execute(sql)
contents = cursor.fetchall()

filtrate = re.compile('[^\u4E00-\u9FA5。！？]')#中文字,字母,句子结束标点
filtered_contents = [filtrate.sub(r' ', content[0]) for content in contents]
filtered_contents


['在白云深处 耸立着威严的太阳神殿。那里很高很高 连老鹰都飞不上去。 虽然很高 但这座宫殿并不寂寞 因为这里住着太阳神和他的女儿  女娃。女娃是个聪明懂事的孩子 太阳神很疼爱她。 每天早上 当女娃还在睡梦中时 太阳神就已经赶到东海 指挥太阳开始一天的工作 直到黄昏 太阳神才能回家。 女娃一个人在家的时候 会经常跑到花园里玩 乖乖地等着爸爸回家。 尽管女娃每天都很开心 但她更希望和爸爸一起去东海 看看太阳升起的地方。 可是 女娃太小了 太阳神不肯带她去 女娃有些失望。  有一天 太阳神回来时发现女娃不开心 就问  乖女儿 你怎么了？  女娃撅着小嘴说  爸爸 我好想去看看外面的世界！  这一次 太阳神答应了女娃 但要等她再长大些才可以去。  日子一天天的过去 女娃实在是等不及了。这天 她趁爸爸不在家 就偷偷地带着小船出去了。 女娃来到东海 看着广阔无边的大海和天空中飞翔的海鸥 心里高兴极了。因为 她还从没见过这么美的风景。 女娃划着小船向东方漂去 突然 海面上刮起了狂风 狂风把沉睡的海魔唤醒了。海魔翻滚着巨浪在海面上横行。 女娃被吓坏了 她使劲划动小船。可是 被海魔卷起的浪花足有几人高 一个巨浪打下来 女娃的小船被打翻了 女娃被卷入了大海中。 女娃被淹没在大海深处 她的灵魂变成了一只鸟 花脑袋 白嘴壳 红爪子。它在海面上翱翔时 会发出 精卫  精卫   的叫声 后来人们就亲切地叫它 精卫 。 精卫痛恨海魔夺去了自己的生命 为了不让更多的人再受到伤害 它决定用衔石填海的方法来对付海魔。 海魔咆哮着 他不理解精卫为什么这么恨它。精卫回答他说  因为你夺去了我的生命 我要用石子把你填成平地 不让你再伤害其他生命。   就这样 精卫日复一日 成年累月地工作 后来它还带领它的孩子们一起衔石填海。 精卫锲而不舍的精神受到了人们的尊敬 后来人们常用 精卫填海  来比喻那些做事意志坚定的人。',
 '  敲敲长 是一根五彩小棍儿 用它敲什么 什么就长大起来。比如 田野里有一棵小豆苗 细细的 矮矮的。米皮皮用 敲敲长 轻轻地敲它 嘴里说 长 长 长  小豆苗就长得又高又壮了。    米皮皮到兔妈妈家串门儿 兔妈妈正在犯愁  唉 胡萝卜刚刚长出地面 孩子们就吵着要吃。 米皮皮说  别急别急 让我来试试  米皮皮用 敲敲长 轻轻地敲着小小的胡萝卜 嘴里说 长 长 长  胡萝卜就长大了 兔

In [148]:
wordLists = []

for content in filtered_contents:
    sentences = re.split("。|！|？", content)
    for sentence in sentences:
        wordList = [word for word in jieba.lcut(sentence)
                    if word != ' ' and word not in ignore_words]
        if len(wordList) > 0:
            wordLists.append(wordList)

In [141]:
word_freq = {}
for wordList in wordLists:
    for word in wordList:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
                
freq_word = []
for word, freq in word_freq.items():
    freq_word.append((word, freq))
freq_word.sort(key = lambda x: x[1], reverse = True)

In [149]:
model = word2vec.Word2Vec(wordLists, min_count=1, size=10)

In [146]:
tot = 0.0
num = 0.0
for i in range(0, 100):
    for j in range(i+1, 100):
        num += 1.0
        tot += model.wv.similarity(freq_word[-i][0], freq_word[-j][0])
tot /= num
tot

-0.0063147861000481783

In [137]:
tots = []

for k in [10, 50, 100, 200, 400]:
    model = word2vec.Word2Vec(wordLists, min_count=3, size=k)
    tot = 0.0
    num = 0.0
    for i in range(0, 300):
        for j in range(i+1, 300):
            num += 1.0
            tot += model.similarity(freq_word[i][0], freq_word[j][0])
    tot /= num
    tots.append(tot)
    
tots

[0.29113075708384328,
 0.37610109553320042,
 0.41539839817808372,
 0.40491119971367173,
 0.41974276980670883]

In [125]:
tots = []

for k in [3, 5, 7]:
    model = word2vec.Word2Vec(wordLists, min_count=3, size=40, window=k)
    tot = 0.0
    num = 0.0
    for i in range(0, 300):
        for j in range(i+1, 300):
            num += 1.0
            tot += model.similarity(freq_word[i][0], freq_word[j][0])
    tot /= num
    tots.append(tot)
    
tots

[0.19895388657644597, 0.39223935848282371, 0.54587342558843621]

In [82]:
for i in range(0, 300):
    print(model[freq_word[i][0]], freq_word[i])

[-3.19264102  5.56178951 -1.91480792  2.59066844 -5.27189732 -4.08232641
  4.16954327 -1.64183605 -2.23253894  0.53465819] ('在', 2648)
[-2.8972044   4.53982687 -1.74987221  2.58167243 -4.34300232 -3.45885468
  3.65684605 -1.40314353 -2.46551132  1.34781849] ('说', 2281)
[-3.21677685  5.00159407 -2.01340914  2.81466293 -5.14415789 -3.86097717
  4.16786194 -1.60131514 -2.73782754  0.94390374] ('把', 1566)
[-2.78201437  5.11185741 -1.75524557  2.45514488 -4.70735979 -3.35393333
  3.57748246 -1.30766118 -2.0714159   0.76700991] ('有', 1304)
[-2.87088251  4.4266901  -1.70183885  2.12251592 -4.4173851  -3.45188522
  3.65506601 -1.46252763 -2.29984117  0.83782965] ('去', 1167)
[-2.86744618  4.88818026 -1.90015244  2.51906347 -4.99685955 -3.83170867
  3.82932734 -1.53378928 -2.57481837  1.03237069] ('又', 1155)
[-3.09168291  4.94738436 -1.96508431  2.74821162 -4.66864967 -3.77698779
  4.12607765 -1.49984694 -2.41086507  1.03668904] ('也', 1153)
[-3.18554091  5.00960541 -1.91729403  2.23078442 -5.186

In [88]:
freq_word[-200:]

[('苦功', 1),
 ('春夏秋冬', 1),
 ('阴晴', 1),
 ('灵感', 1),
 ('铺纸', 1),
 ('研墨', 1),
 ('印在', 1),
 ('画案', 1),
 ('一站', 1),
 ('形态各异', 1),
 ('画竹', 1),
 ('自信', 1),
 ('晁', 1),
 ('补之对文', 1),
 ('竹时', 1),
 ('胸中', 1),
 ('成竹', 1),
 ('充分准备', 1),
 ('睢宁', 1),
 ('长者', 1),
 ('迟到', 1),
 ('鸡刚', 1),
 ('张良来', 1),
 ('兵书', 1),
 ('开国功臣', 1),
 ('孺子', 1),
 ('离群', 1),
 ('山丘', 1),
 ('向天', 1),
 ('那贼', 1),
 ('大牛', 1),
 ('逃离', 1),
 ('吓破', 1),
 ('胆', 1),
 ('誓言', 1),
 ('烈日', 1),
 ('梧桐树', 1),
 ('阔大', 1),
 ('大发议论', 1),
 ('之树', 1),
 ('咬人', 1),
 ('上摇着', 1),
 ('隐秘', 1),
 ('人装', 1),
 ('毫不迟疑', 1),
 ('进网', 1),
 ('冠雀时', 1),
 ('疗效', 1),
 ('恶性', 1),
 ('事实证明', 1),
 ('强得', 1),
 ('整捆', 1),
 ('竭尽', 1),
 ('打垮', 1),
 ('内讧', 1),
 ('耗损', 1),
 ('伤势', 1),
 ('消遣', 1),
 ('阿提卡', 1),
 ('苏尼翁', 1),
 ('海峡', 1),
 ('钻到', 1),
 ('托起来', 1),
 ('送往', 1),
 ('海港', 1),
 ('欧斯时', 1),
 ('海琢', 1),
 ('知不知道', 1),
 ('欧斯', 1),
 ('上药', 1),
 ('顺手牵羊', 1),
 ('家具', 1),
 ('偷光', 1),
 ('更糟', 1),
 ('罪证', 1),
 ('妈想', 1),
 ('说大话', 1),
 ('夸海口', 1),
 ('暴露', 1),
 ('布下', 1),
 ('那末', 1),


In [138]:
freq_word[0:300]

[('在', 180),
 ('说', 115),
 ('把', 95),
 ('小', 85),
 ('有', 82),
 ('里', 77),
 ('到', 70),
 ('也', 70),
 ('人', 69),
 ('又', 65),
 ('上', 62),
 ('不', 61),
 ('这', 60),
 ('去', 56),
 ('来', 47),
 ('从', 41),
 ('被', 40),
 ('用', 39),
 ('给', 39),
 ('大', 38),
 ('要', 37),
 ('很', 34),
 ('自己', 34),
 ('一只', 33),
 ('让', 32),
 ('呢', 32),
 ('得', 31),
 ('长', 30),
 ('好', 29),
 ('还', 28),
 ('这个', 28),
 ('对', 28),
 ('小兔', 25),
 ('做', 25),
 ('大家', 25),
 ('牛顿', 25),
 ('爸爸', 24),
 ('吧', 24),
 ('一', 24),
 ('向', 23),
 ('变成', 23),
 ('公主', 23),
 ('国王', 23),
 ('于是', 23),
 ('为', 23),
 ('孩子', 22),
 ('怎么', 22),
 ('想', 22),
 ('中', 22),
 ('叫', 21),
 ('这样', 21),
 ('什么', 21),
 ('小狗', 21),
 ('下', 21),
 ('就是', 21),
 ('帽子', 21),
 ('看', 21),
 ('晚安', 21),
 ('个', 20),
 ('人们', 20),
 ('年', 20),
 ('知道', 20),
 ('盘古', 20),
 ('一天', 19),
 ('会', 19),
 ('呀', 19),
 ('能', 19),
 ('东西', 19),
 ('俏俏', 19),
 ('可', 19),
 ('榨菜', 19),
 ('火', 19),
 ('起来', 18),
 ('吃', 18),
 ('听', 18),
 ('女娃', 17),
 ('可是', 17),
 ('礼物', 17),
 ('再', 16),
 ('可以', 16),
 ('原来',

In [122]:
wordLists[:100]

[['在', '白云', '深处', '耸立着', '威严', '太阳神', '殿'],
 ['那里', '很', '高', '很', '高', '连', '老鹰', '飞', '不', '上去'],
 ['虽然',
  '很',
  '高',
  '但',
  '这座',
  '宫殿',
  '并',
  '不',
  '寂寞',
  '因为',
  '这里',
  '住',
  '太阳神',
  '女儿',
  '女娃'],
 ['女娃', '个', '聪明', '懂事', '孩子', '太阳神', '很', '疼爱'],
 ['每天',
  '早上',
  '当',
  '女娃',
  '还',
  '在',
  '睡梦中',
  '时',
  '太阳神',
  '已经',
  '赶到',
  '东海',
  '指挥',
  '太阳',
  '开始',
  '一天',
  '工作',
  '直到',
  '黄昏',
  '太阳神',
  '才能',
  '回家'],
 ['女娃',
  '人',
  '在家',
  '时候',
  '会',
  '经常',
  '跑',
  '到',
  '花园里',
  '玩',
  '乖乖',
  '等',
  '爸爸',
  '回家'],
 ['尽管',
  '女娃',
  '每天',
  '很',
  '开心',
  '但',
  '更',
  '希望',
  '爸爸',
  '一起',
  '去',
  '东海',
  '看看',
  '太阳',
  '升起',
  '地方'],
 ['可是', '女娃', '太小', '太阳神', '不肯', '带', '去', '女娃', '有些', '失望'],
 ['有', '一天', '太阳神', '回来', '时', '发现', '女娃', '不', '开心', '问', '乖', '女儿', '怎么'],
 ['女娃', '撅', '小', '嘴', '说', '爸爸', '好', '想', '去', '看看', '外面', '世界'],
 ['这', '一次', '太阳神', '答应', '女娃', '但', '要', '等', '再', '长大', '些', '才', '可以', '去'],
 ['日子', '一天天', '过去', '女娃', '实在', '等不及

In [153]:
model.n_similarity(['色彩'],['比如']) 

0.94908172675427882

In [155]:
words = ''
for wordList in wordLists:
    for word in wordList:
        words += word + ' '

words

'在 白云 深处 耸立着 威严 太阳神 殿 那里 很 高 很 高 连 老鹰 飞 不 上去 虽然 很 高 但 这座 宫殿 并 不 寂寞 因为 这里 住 太阳神 女儿 女娃 女娃 个 聪明 懂事 孩子 太阳神 很 疼爱 每天 早上 当 女娃 还 在 睡梦中 时 太阳神 已经 赶到 东海 指挥 太阳 开始 一天 工作 直到 黄昏 太阳神 才能 回家 女娃 人 在家 时候 会 经常 跑 到 花园里 玩 乖乖 等 爸爸 回家 尽管 女娃 每天 很 开心 但 更 希望 爸爸 一起 去 东海 看看 太阳 升起 地方 可是 女娃 太小 太阳神 不肯 带 去 女娃 有些 失望 有 一天 太阳神 回来 时 发现 女娃 不 开心 问 乖 女儿 怎么 女娃 撅 小 嘴 说 爸爸 好 想 去 看看 外面 世界 这 一次 太阳神 答应 女娃 但 要 等 再 长大 些 才 可以 去 日子 一天天 过去 女娃 实在 等不及 这天 趁 爸爸 不 在家 偷偷 地带 小船 出去 女娃 来到 东海 看着 广阔 无边 大海 天空 中 飞翔 海鸥 心里 高兴 极了 因为 还 从没 见过 这么 美的 风景 女娃 划着 小船 向 东方 漂去 突然 海面 上 刮起 狂风 狂风 把 沉睡 海魔 唤醒 海魔 翻滚 巨浪 在 海面 上 横行 女娃 被 吓坏 使劲 划动 小船 可是 被 海魔 卷起 浪花 足有 几人 高 巨浪 打 下来 女娃 小船 被 打翻 女娃 被 卷入 大海 中 女娃 被 淹没 在 大海 深处 灵魂 变成 一只 鸟 花 脑袋 白嘴 壳 红 爪子 在 海面 上 翱翔 时 会 发出 精卫 精卫 叫声 后来 人们 亲切 叫 精卫 精卫 痛恨 海魔 夺去 自己 生命 为了 不让 更 多 人 再 受到 伤害 决定 用 衔石填海 方法 来 对付 海魔 海魔 咆哮 不 理解 精卫 为什么 这么 恨 精卫 回答 说 因为 夺去 生命 我要 用 石子 把 填成 平地 不让 再 伤害 其他 生命 这样 精卫 日复一日 成年累月 工作 后来 还 带领 孩子 一起 衔石填海 精卫 锲而不舍 精神 受到 人们 尊敬 后来 人们 常用 精卫填海 来 比喻 那些 做事 意志 坚定 人 敲敲 长 一根 五彩 小 棍儿 用 敲 什么 什么 长大 起来 比如 田野 里 有 一棵 小 豆苗 细细的 矮矮的 米 皮皮 用 敲敲 长 轻轻地 敲

In [156]:
f = open('train_data','w')
f.write(words)
f.close()

In [157]:
len(freq_word)

6133